In [1]:
import tensorflow.keras
from unet import unet_model_3d


In [2]:
input_shape = (1,512,512,95)
model = unet_model_3d(input_shape = input_shape, n_labels = 2, activation_name = 'relu')
#model.summary()

In [3]:
from data import create_dataset

batch_size = 5

train_dataset = create_dataset(batch_size,'/home/jaume/Documentos/DL_postgraduate/final_project/sample_data/resized_training/')

In [4]:
#for image in train_dataset:
#    print(image.shape)

In [5]:
model.fit(train_dataset,epochs=2,verbose=1)

Epoch 1/2
      1/Unknown - 0s 28ms/step

IndexError: tuple index out of range